In [2]:
import pandas as pd
import os

# 1. 파일 경로 설정
file_path = '서울교통공사_1_8호선 역사 좌표(위경도) 정보_20250814.csv'

# 2. CSV 파일 로드
try:
    df_raw = pd.read_csv(file_path, encoding='utf-8')
except:
    df_raw = pd.read_csv(file_path, encoding='cp949')

print("CSV 파일 로드 성공")

# 3. 데이터 전처리 (Transform)
# (1) 컬럼명 매핑
rename_map = {
    '고유역번호(외부역코드)': 'stnCd',
    '역명': 'stnNm',
    '위도': 'lat',
    '경도': 'lon'
}
df_raw.rename(columns=rename_map, inplace=True)

# (2) 호선명 포맷 통일 (1 -> 1호선)
# 기존 DB의 lineNm 컬럼과 조인하기 위해 필수적인 단계입니다.
if '호선' in df_raw.columns:
    # 1. 결과를 담을 빈 리스트를 만듭니다.
    new_lines = []

    # 2. '호선' 컬럼에 있는 데이터를 하나씩 꺼내서 확인합니다.
    for x in df_raw['호선']:
        
        # 일단 안전하게 문자로 바꿉니다. (astype(str) 역할)
        x = str(x)
        new_lines.append(x + '호선')

    # 3. 다 만든 리스트를 새로운 컬럼 'lineNm'에 집어넣습니다.
    df_raw['lineNm'] = new_lines
else:
    df_raw['lineNm'] = '정보없음'

# (3) 역코드 문자열 변환 (DB 타입 매칭)
df_raw['stnCd'] = df_raw['stnCd'].astype(str).str.strip()

# (4) 최종 컬럼 선택
target_cols = ['stnCd', 'stnNm', 'lineNm', 'lat', 'lon']
df_final = df_raw[target_cols].copy()

# 4. 결과 확인 (상위 5개)
print("\n[변환 결과 미리보기] station_meta 테이블 적재용")
print("="*60)

display(df_final.head(5))
display(df_final.info())

print("-" * 60)
print(f"총 데이터 건수: {len(df_final)}개")

CSV 파일 로드 성공

[변환 결과 미리보기] station_meta 테이블 적재용


,stnCd,stnNm,lineNm,lat,lon
0,150,서울,1호선,37.553150,126.972533
1,151,시청,1호선,37.563590,126.975407
2,152,종각,1호선,37.570203,126.983116
3,153,종로3가,1호선,37.570429,126.992095
4,154,종로5가,1호선,37.570971,127.001900


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276 entries, 0 to 275
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   stnCd   276 non-null    object 
 1   stnNm   276 non-null    object 
 2   lineNm  276 non-null    object 
 3   lat     276 non-null    float64
 4   lon     276 non-null    float64
dtypes: float64(2), object(3)
memory usage: 10.9+ KB


None

------------------------------------------------------------
총 데이터 건수: 276개


In [3]:
import pymysql
import dbconnect

# ==========================================
# [Step 5] DB 연결 및 데이터 적재
# ==========================================

conn = dbconnect.MydbConnect('seoul_urban_lab',port=3306)
# ⚠️ 주의: conn (DB 연결 객체)은 이미 위에서 만들어져 있다고 가정합니다.
# 만약 연결이 끊겼다면 다시 연결하는 코드가 필요합니다.
cursor = conn.cursor()

# 1. SQL 쿼리 작성
# (이미 있는 역이라면 정보를 최신으로 갱신하는 'ON DUPLICATE KEY UPDATE' 구문 사용)
sql = """
INSERT INTO station_meta
(stnCd, stnNm, lineNm, lat, lon)
VALUES (%s, %s, %s, %s, %s)
ON DUPLICATE KEY UPDATE
    stnNm = VALUES(stnNm),
    lineNm = VALUES(lineNm),
    lat = VALUES(lat),
    lon = VALUES(lon)
"""

# 2. 데이터프레임 값을 리스트로 변환
data_rows = df_final.values

count = 0
print(f"\n[DB 적재 시작] 총 {len(data_rows)}건 처리 예정...")

# 3. 한 줄씩 꺼내서 저장 (안전한 방식)
for row in data_rows:
    try:
        # (1) 변수 추출 및 형변환 (Type Casting)
        # DB에 들어갈 때 타입 에러가 나지 않도록 명확하게 변환해줍니다.
        stn_cd = str(row[0])      # stnCd (문자)
        stn_nm = str(row[1])      # stnNm (문자)
        line_nm = str(row[2])     # lineNm (문자)
        lat = float(row[3])       # lat (실수 - 위도)
        lon = float(row[4])       # lon (실수 - 경도)

        # (2) 튜플로 묶기
        data = (stn_cd, stn_nm, line_nm, lat, lon)

        # (3) 쿼리 실행
        cursor.execute(sql, data)
        count += 1

    except Exception as e:
        # 에러가 나면 어떤 역에서 났는지 알려줌
        print(f"에러: {e}")

# 4. 최종 커밋 (저장 확정)
try:
    conn.commit()
    print("=" * 60)
    print(f"{count}건이 station_meta 테이블에 들어갔습니다")
    print("=" * 60)
except Exception as e:
    conn.rollback() # 실수했으면 되돌리기
    print(f"에러: {e}")

데이터베이서 연결 시작!
MySQL DB 연결 성공

[DB 적재 시작] 총 276건 처리 예정...
276건이 station_meta 테이블에 들어갔습니다
